<center>
<h2>Preliminaries</h2>
</center>

<h3>Structured prediction</h3>

<table  style="border-style: hidden; border-collapse: collapse; padding: 50px">
<thead>
<tr>
<th>I</th> 
<th>studied</th>
<th>in</th>
<th>London</th>
<th>with</th>
<th>Sebastian</th>
<th>Riedel</th>
</tr>
</thead>
<tbody style="font-size:100%">
<tr>
<td><span class="fragment" data-fragment-index="1">PRP</span></td>
<td><span class="fragment" data-fragment-index="1">VBD</span></td>
<td><span class="fragment" data-fragment-index="1">IN</span></td>
<td><span class="fragment" data-fragment-index="1">NNP</span></td>
<td><span class="fragment" data-fragment-index="1">IN</span></td>
<td><span class="fragment" data-fragment-index="1">NNP</span></td>
<td><span class="fragment" data-fragment-index="1">NNP</span></td>
</tr>
<tr>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">B-LOC</span></td>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">B-PER</span></td>
<td><span class="fragment" data-fragment-index="2">I-PER</span></td>
</tr>
</tbody>
</table>

<p>
				<ul>
  			<li class="fragment" data-fragment-index="1">part of speech (PoS) tagging</li>
  			<li class="fragment" data-fragment-index="2">named entity recognition (NER)</li>
				</ul>
			</p>


<p><b>Input:</b> a sentence $\mathbf{x}=[x_1...x_N]$<br> <b>Output:</b> a sequence of labels $\mathbf{y}=[y_{1}\ldots y_{N}] \in {\cal Y}^N$</p>

<h3>Structured prediction</h3>

<img src="images/tikz/semParse.png" style="width:100%;">

<p>Semantic parsing, but also syntactic parsing, semantic role labeling, question answering over knowledge bases, etc.)</p>
<p><b>Input:</b> a sentence $\mathbf{x}=[x_1...x_N]$<br>
<b>Output:</b> a meaning representation graph $\mathbf{G}=(V,E) \in {\cal G_{\mathbf{x}}}$</p> 

<h3>Structured prediction</h3>

<img src="images/nlg.png" style="width:90%;">

<p>Natural language generation (NLG), but also summarization, decoding in machine translation, etc.</p>

<p><b>Input:</b> a meaning representation<br>
<b>Output:</b> $\mathbf{w}=[w_1...w_N], w\in {\cal V}\cup END, w_N=END$</p>  

### Two main paradigms

Joint modeling, a.k.a: 
- global inference
- structured models

Incremental modeling, a.k.a:
- local 
- greedy
- pipeline
- transition-based
- history-based

### Joint modeling

A model (e.g. conditional random field) that scores complete outputs (e.g. label sequences):

$$\mathbf{\hat y} =\hat y_{1}\ldots \hat y_{N} = \mathop{\arg \max}_{Y \in {\cal Y}^N} f(y_{1}\ldots y_{N}, \mathbf{x})$$

<ul class="fragment">
					<li>exhaustive exploration of the search space</li>
					<li>large/complex search spaces are challenging</li>
					<li>efficient dynamic programming restricts modelling flexibility
						(i.e. Markov assumptions)</li>
				</ul>


### Incremental modeling

A classifier predicting a label at a time given the previous ones:


\begin{align}
\hat y_1 &=\mathop{\arg \max}_{y \in {\cal Y}} f(y, \mathbf{x}),\\
\mathbf{\hat y} = \quad \hat y_2 &=\mathop{\arg \max}_{y \in {\cal Y}} f(y, \mathbf{x}, \hat y_1), \cdots\\
\hat y_N &=\mathop{\arg \max}_{y \in {\cal Y}} f(y, \mathbf{x}, \hat y_{1} \ldots \hat y_{N-1})
\end{align}

<ul class="fragment">
					<li>use our favourite classifier</li>
					<li>no restrictions on features</li>
					<li>prone to error propagation (i.i.d. assumption broken)</li>
					<li>local model not trained wrt the task-level loss</li>
				</ul>


### Imitation learning

Improve incremental modeling to:
- address error-propagation
- train wrt the task-level loss function

**Meta-learning**: use our favourite classifier and features,
but generate better (non-i.i.d.) training data

To apply IL we need:
- transition system (what our classifier can do)
- task loss (what we optimize for)
- expert policy (the teacher to help us)

<h3>Transition system</h3>

<p>The <b>actions</b> $\cal A$ the classifier $f$ can predict and their effect on the <b>state</b> which tracks the prediction: $S_{t+1}=S_1(\alpha_1\ldots\alpha_t)$</p>

<img src="images/tikz/IncrementalStructure.png" style="align:center; width:65%">

<h3>Transition system</h3>

<p style="text-align: left; border:3px; border-radius: 25px; background-color:lightgrey; border-style:solid; border-color:black; padding: 0.3em; font-size: 75%">
\begin{align}
& \textbf{Input:} \; sentence \; \mathbf{x}\\
& state \; S_1=initialize(\mathbf{x}); timestep \; t = 1\\
& \mathbf{while}\; S_t \; \text{not final}\; \mathbf{do}\\
& \quad action \; \alpha_t = \mathop{\arg \max}_{\alpha \in {\cal A}} f(\alpha, \mathbf{x})\\
& \quad S_{t+1}=S_t(\alpha_t); t=t+1\\
\end{align}
</p>

<ul>
<li><b>PoS/NER tagging?</b> <span class="fragment">for each word, left-to-right, predict a PoS tag which is added to the output</span></li>
<li class="fragment"><b>NLG?</b> <span class="fragment">predict a word adding it to the output until the <code>EndOfSentence</code> is predicted</span></li>
</ul>

### Supervising the classifier

What are good actions in incremental structured prediction?

Those that reach $S_{final} = S_1(\alpha_1\ldots\alpha_T)$ with low **task loss**:

$$loss  = L(S_{final}, \mathbf{y}) \geq 0$$

<ul>
<li><b>PoS tagging?</b> <span class="fragment">Hamming loss: number of incorrect tags</span></li>
<li class="fragment"><b>NER?</b> <span class="fragment">number of false positives and false negatives</span></li>
<li class="fragment"><b>NLG?</b> <span class="fragment">BLEU: % of n-grams predicted present in the gold reference(s), i.e. $L=1-BLEU(S_{final}, \mathbf{y})$</span></li>
</ul>

### Action assessment 

<table style="font-size:80%; border-style: hidden; border-collapse: collapse; padding: 50px">
<thead>
<tr>
<th>I</th> 
<th>studied</th>
<th>in</th>
<th>London</th>
<th>with</th>
<th>Sebastian</th>
<th>Riedel</th>
</tr>
</thead>
<tbody>
<tr>
<td>PRP</td>
<td>VBD</td>
<td>IN</td>
<td>NNP</td>
<td>IN</td>
<td>NNP</td>
<td><span class="fragment" data-fragment-index="1">NNP</span></td>
</tr>
<tr>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">B-LOC</span></td>
<td><span class="fragment" data-fragment-index="2">O</span></td>
<td><span class="fragment" data-fragment-index="2">B-PER</span></td>
<td><span class="fragment" data-fragment-index="3">I-PER</span></td>
</tr>
</tbody>
</table>

<p>How many incorrect PoS tags due to $\alpha_6$  being NNP? <span class="fragment" data-fragment-index="1"><b>0</b></span>
</p>
<p class="fragment" data-fragment-index="2"> How many $FP+FN$ due to $\alpha_6$ being B-PER? <span class="fragment" data-fragment-index="3"><br><b>Depends!</b> If $\alpha_7$ is</span>
<ul class="fragment" data-fragment-index="3">
<li>I-PER:  $0$ (correct)</li> 
<li>O: $2$ (1FP+1FN)</li>
<li>B-*: $3$ (2FP+1FN)</li>
</ul>
</p>

### More action assessment

Can we tell how predicting a word will change our BLEU?
$$
BLEU([\alpha_1...\alpha_T],\mathbf{y}) = \prod_{n=1}^N \frac{\# \text{n-grams} \in ([\alpha_1...\alpha_T] \cap \mathbf{y})}{\# \text{n-grams} \in [\alpha_1..\alpha_T]}
$$
<p class="fragment" data-fragment-index="1"><b>No</b>! (assuming N>1)</p>

**Non-decomposable** losses wrt transition system are common!

Imitation learning trains incremental models for such losses

Affects joint models too: loss does not always decompose over the graphical model ([Tarlow and Zemel, 2012](http://www.cs.toronto.edu/~dtarlow/tarlow_zemel_aistats12.pdf))


Even F-score for binary classification is non-decomposable ([Narasimhan et al., 2015](http://jmlr.org/proceedings/papers/v37/narasimhana15.pdf))!

### Expert policy

Returns the best action at the current state by looking at the gold standard assuming **future actions are also optimal**:

$$\alpha^{\star}=\pi^{\star}(S_t, \mathbf{y}) = \mathop{\arg \min}_{\alpha \in {\cal A}} L(S_t(\alpha,\pi^{\star}),\mathbf{y})$$

<table style="border-style: hidden; border-collapse: collapse; padding: 50px">
<thead>
<tr>
<th>I</th> 
<th>studied</th>
<th>in</th>
<th>London</th>
<th>with</th>
<th>Sebastian</th>
<th>Riedel</th>
</tr>
</thead>
<tbody>
<tr>
<td>PRP</td>
<td>VBD</td>
<td>IN</td>
<td>NNP</td>
<td>IN</td>
<td>NNP</td>
<td>NNP</td>
</tr>
</tbody>
</table>

<p>PoS tagging: $\pi^{\star}(S_t, \mathbf{y})= \pi^{\star}(S_t, [y_1...y_T])=y_t$</p>

<p style="float: left;">Only available for the training data: an expert<br>demonstrating how to perform the task </p> <a href="http://www.salon.com/2016/10/06/what-makes-a-good-teacher-why-certifications-and-standards-dont-guarantee-quality-educators_partner/"><img src="images/english_teacher.jpg" style="width:20%; float: right;"></a>

### Expert policy

What action should $\pi^{\star}$ return?

<table style="border-style: hidden; border-collapse: collapse; padding: 50px">
<thead>
<tr>
<th>I</th> 
<th>studied</th>
<th>in</th>
<th>London</th>
<th>with</th>
<th>Sebastian</th>
<th>Riedel</th>
</tr>
</thead>
<tbody>
<tr>
<td>O</td>
<td>O</td>
<td>O</td>
<td>B-LOC</td>
<td>O</td>
<td>B-PER</td>
<td><span class="fragment" data-fragment-index="1">I-PER</span></td>
</tr>
<tr class="fragment" data-fragment-index="2">
<td>O</td>
<td>O</td>
<td>O</td>
<td>B-LOC</td>
<td>O</td>
<td>O</td>
<td><span class="fragment" data-fragment-index="3">O</span></td>
</tr>
</tbody>
</table> 

Takes previous actions into account to be optimal (dynamic)

Finding the optimal action can be expensive.

IL can learn with sub-optimal experts!

### Cost-sensitive classification

Each action has a different cost:

<table style="border-style: hidden; border-collapse: collapse; padding: 50px">
<thead>
<tr>
<th>O</th> 
<th>B-PER</th>
<th>I-PER</th>
<th>B-LOC</th>
<th>I-LOC</th>
<th>B-ORG</th>
<th>I-LOC</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>2</td>
<td>2</td>
<td>0</td>
<td>1</td>
<td>1</td>
<td>1</td>
</tr>
</tbody>
</table>

Learning with costs:
- sample instances according to their cost to train any classifier ([Abe et al., 2004](http://www.hunch.net/~jl/projects/reductions/mc2/p542-Abe.pdf))
- in error-driven learning adjust the updates in proportion to the error cost ([Crammer et al., 2006](http://jmlr.csail.mit.edu/papers/volume7/crammer06a/crammer06a.pdf))

**Multi-label classification**: multiple correct labels per-instance

**Cost-sensitive classification**: 

### Imitation learning

Learn an action-predicting classifier by imitating expert demonstrations
- learn that some mistakes are worse than others
- multiple actions can be optimal